In [1]:
import os
import cv2
from glob import glob
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm # used for creating progress bars
import shutil
%matplotlib inline
import laso_mask_builder as lmb
from datetime import datetime
import grid_strategy as gs
import patient_wise_split as pws

#Deep learning libraries
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers.legacy import Adam
from keras import Model
from keras.models import load_model
from keras.layers import Dense,Dropout,Flatten,BatchNormalization
from tensorflow.python.keras import backend as KB

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

#PIL version should be 8.2.0 only
import PIL
print(PIL.__version__)

from platform import python_version
print(python_version() )
gpu = len(tf.config.list_physical_devices('GPU'))>0
print(tf.config.list_physical_devices('GPU'))

print(KB._get_available_gpus())

10.2.0
3.10.8
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
['/device:GPU:0']


In [2]:
def check_split(train_labels, test_labels, val_labels, val_patient_ids):
    if len(val_patient_ids) < 6:
        return False

    train_positive = train_labels.count('0_positive')
    train_negative = train_labels.count('2_negative')
    test_positive = test_labels.count('0_positive')
    test_negative = test_labels.count('2_negative')
    val_positive = val_labels.count('0_positive')
    val_negative = val_labels.count('2_negative')

    if train_positive == 0 or train_negative == 0 or test_positive == 0 or test_negative == 0 or val_positive == 0 or val_negative == 0:
        return False

    train_balance_ratio = train_positive/train_negative if train_negative > train_positive else train_negative/train_positive
    test_balance_ratio = test_positive/test_negative if test_negative > test_positive else test_negative/test_positive
    val_balance_ratio = val_positive/val_negative if val_negative > val_positive else val_negative/val_positive
    test_val_count_ratio = len(test_labels)/len(val_labels) if len(val_labels) > len(test_labels) else len(val_labels)/len(test_labels)

    if train_balance_ratio < 0.3 or test_balance_ratio < 0.3 or val_balance_ratio < 0.3 or test_val_count_ratio < 0.6:
        return False
    else:
        return True

#PWS Image Loading and Splitting
positive_directory = './ak-dataset/ak/0_positive'
negative_directory = './ak-dataset/ak/2_negative'

while True:
    train_paths, test_paths, val_paths = pws.train_test_val_split(0.7, 0.15, 0.15, positive_directory, negative_directory)
    val_patient_ids = set([path.split('/')[-1].split('_')[0] for path in val_paths])

    train_labels = [path.split('/')[-2] for path in train_paths]
    test_labels = [path.split('/')[-2] for path in test_paths]
    val_labels = [path.split('/')[-2] for path in val_paths]
    print('Train length:',len(train_paths), 'Positive:', train_labels.count('0_positive'), 'Negative:', train_labels.count('2_negative'), '\n',
          'Test length:',len(test_paths), 'Positive:', test_labels.count('0_positive'), 'Negative:', test_labels.count('2_negative'), '\n',
          'Val length:',len(val_paths), 'Positive:', val_labels.count('0_positive'), 'Negative:', val_labels.count('2_negative'), 'Patients:', len(val_patient_ids), '\n')
    if check_split(train_labels, test_labels, val_labels, val_patient_ids):
        break

# Create DataFrames
X_train = pd.DataFrame({'filepath' : train_paths, 'category' : train_labels})
X_test = pd.DataFrame({'filepath' : test_paths, 'category' : test_labels})
X_val = pd.DataFrame({'filepath' : val_paths, 'category' : val_labels})

# Creating column with the type of data i.e train or val
X_train['type'] = 'train'
X_test['type'] = 'test'
X_val['type'] = 'val'

manual_mask_paths = []
manual_mask_paths = glob('./ak-dataset/ak_jpg/0_positive_cropped/*')

Train length: 3047 Positive: 1066 Negative: 1981 
 Test length: 248 Positive: 196 Negative: 52 
 Val length: 1104 Positive: 52 Negative: 1052 Patients: 15 

Train length: 3008 Positive: 409 Negative: 2599 
 Test length: 587 Positive: 391 Negative: 196 
 Val length: 804 Positive: 514 Negative: 290 Patients: 10 

Train length: 3767 Positive: 905 Negative: 2862 
 Test length: 0 Positive: 0 Negative: 0 
 Val length: 632 Positive: 409 Negative: 223 Patients: 12 

Train length: 3321 Positive: 743 Negative: 2578 
 Test length: 411 Positive: 411 Negative: 0 
 Val length: 667 Positive: 160 Negative: 507 Patients: 10 

Train length: 3739 Positive: 1314 Negative: 2425 
 Test length: 12 Positive: 0 Negative: 12 
 Val length: 648 Positive: 0 Negative: 648 Patients: 2 

Train length: 3143 Positive: 832 Negative: 2311 
 Test length: 591 Positive: 158 Negative: 433 
 Val length: 665 Positive: 324 Negative: 341 Patients: 12 



In [3]:
def process_images_from_df(dataframe, path, manual_paths, lasso=False, msr=False):
    positive_count = 0
    negatives = []
    for i, row in tqdm(dataframe.iterrows(), total=len(dataframe)):
        # Positive image, load manual cropped original
        image_name = row['filepath'].split('/')[-1].split('.')[0]
        if row['category'] == '0_positive':
            for m_path in manual_paths:
                mask_name = '_'.join(m_path.split('/')[-1].split('_')[:-1])
                if image_name == mask_name:
                    positive_count += 1
                    image = cv2.imread(m_path, cv2.IMREAD_GRAYSCALE)
                    if lasso:
                        mask = lmb.flood_fill(image, seed=(image.shape[0]//2, image.shape[1]//2))
                        image = lmb.overlay_mask_and_original(image, mask)
                    cv2.imwrite(path + '/positive/' + m_path.split('/')[-1], image)
        else:
            # Negative image, create all negative mask
            image = cv2.imread(row['filepath'], cv2.IMREAD_GRAYSCALE)
            if msr:
                image_MSR = lmb.msr(image)
                seeds = lmb.find_seed_points(image_MSR)
            else:
                seeds = lmb.find_seed_points(image)
            if len(seeds) > 0:
                for seed in seeds:
                    if lasso:
                        mask = lmb.flood_fill(image, seed=seed, lower=50, upper=50)
                        composite = lmb.overlay_mask_and_original(image, mask)
                    composite = lmb.crop_100x100_from_point(composite, seed, adaptive_pad_color=False)
                    negatives.append(composite)

    print(len(negatives), "picking", positive_count)
    negatives = random.sample(negatives, positive_count)
    for i, crop in enumerate(negatives):
        if lasso:
            mask = lmb.flood_fill(crop, seed=(crop.shape[0]//2, crop.shape[1]//2))
            # crop = lmb.overlay_mask_and_original(crop, mask)
        cv2.imwrite(path + '/negative/' + str(i) + '.jpg', crop)

# Make mask directories
if not os.path.exists('./masks_temp'):
    os.makedirs('./masks_temp')
    os.makedirs('./masks_temp/train')
    os.makedirs('./masks_temp/train/negative')
    os.makedirs('./masks_temp/train/positive')
    os.makedirs('./masks_temp/test')
    os.makedirs('./masks_temp/test/negative')
    os.makedirs('./masks_temp/test/positive')
    os.makedirs('./masks_temp/val')
shutil.rmtree('./masks_temp/train/negative')
shutil.rmtree('./masks_temp/train/positive')
shutil.rmtree('./masks_temp/test/negative')
shutil.rmtree('./masks_temp/test/positive')
shutil.rmtree('./masks_temp/val')
os.makedirs('./masks_temp/train/negative')
os.makedirs('./masks_temp/train/positive')
os.makedirs('./masks_temp/test/negative')
os.makedirs('./masks_temp/test/positive')
os.makedirs('./masks_temp/val')

train_folder = './masks_temp/train'
test_folder = './masks_temp/test'

process_images_from_df(X_train, train_folder, manual_mask_paths, lasso=True, msr=False)
process_images_from_df(X_test, test_folder, manual_mask_paths, lasso=True, msr=False)

100%|██████████| 3143/3143 [00:27<00:00, 116.24it/s]


73706 picking 1719


100%|██████████| 591/591 [00:04<00:00, 125.30it/s]


15372 picking 467


In [4]:
target_width=100
target_height=100

#training data
train_datagen = ImageDataGenerator(rescale = 1.0/255.,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   rotation_range=90,
                                #    height_shift_range = 0.5,
                                #    width_shift_range = 0.5,
                               )

train_generator = train_datagen.flow_from_directory(train_folder,
                                                    batch_size=64,
                                                    shuffle=True,
                                                    # color_mode = "grayscale",
                                                    class_mode='binary',
                                                    target_size=(target_width,target_height))

#testing data
test_datagen = ImageDataGenerator(rescale = 1.0/255.,
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  rotation_range=90,
                                #   height_shift_range = 0.5,
                                #   width_shift_range = 0.5,
                                  )

test_generator = test_datagen.flow_from_directory(test_folder,
                                                  batch_size=64,
                                                  shuffle=False,
                                                  # color_mode = "grayscale",
                                                  class_mode='binary',
                                                  target_size=(target_width,target_height))

print(train_generator.image_shape)

Found 3438 images belonging to 2 classes.
Found 934 images belonging to 2 classes.
(100, 100, 3)


In [5]:
callbacks = EarlyStopping(monitor="val_loss",patience=7,verbose=1,mode='auto')
now = datetime.now()
best_model_file = './models/ak_resnet_model_' + now.strftime("%y%j%H%M%S") + '.h5'
best_model = ModelCheckpoint(best_model_file, mode='max',monitor='val_precision',save_best_only=True,verbose=1)

res_base = tf.keras.applications.resnet_v2.ResNet101V2(weights='imagenet',include_top=False,input_shape=(target_width,target_height,3),input_tensor=None)

res_base.trainable = False

output = res_base.get_layer(index=-1).output
output = Flatten()(output)
output = Dense(256, activation='relu')(output)
output = BatchNormalization()(output) 
output = Dropout(0.4)(output)
output = Dense(128,activation='relu')(output)
output = BatchNormalization()(output)
output = Dropout(0.3)(output)
output = Dense(64,activation='relu')(output)
output = Dropout(0.3)(output) 
output = Dense(32,activation='relu')(output)
output = BatchNormalization()(output)
output = Dense(1, activation='sigmoid')(output)
print("New layers are added")

resnet_model = Model(res_base.input,output)

resnet_model.summary()

New layers are added
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 100, 100, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 106, 106, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 50, 50, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 pool1_pad (ZeroPadding2D)   (None, 52, 52, 64)           0         ['conv1_conv[0][0]']          
                                                                         

In [6]:
resnet_model.compile(optimizer=Adam(learning_rate=0.0001),loss='binary_crossentropy',metrics = [keras.metrics.Precision(name='precision')])
history = resnet_model.fit(train_generator,epochs=50,validation_data=test_generator,callbacks=[callbacks,best_model],verbose=1)

Epoch 1/50
54/54 [==============================] - ETA: 0s - loss: 0.4210 - precision: 0.8598
Epoch 1: val_precision improved from -inf to 0.66214, saving model to ./ak_resnet_model_24138194854.h5


/opt/homebrew/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


54/54 [==============================] - 77s 988ms/step - loss: 0.4210 - precision: 0.8598 - val_loss: 0.6002 - val_precision: 0.6621
Epoch 2/50
54/54 [==============================] - ETA: 0s - loss: 0.3386 - precision: 0.9003
Epoch 2: val_precision improved from 0.66214 to 0.80651, saving model to ./ak_resnet_model_24138194854.h5
54/54 [==============================] - 9s 166ms/step - loss: 0.3386 - precision: 0.9003 - val_loss: 0.4304 - val_precision: 0.8065
Epoch 3/50
54/54 [==============================] - ETA: 0s - loss: 0.3151 - precision: 0.9076
Epoch 3: val_precision improved from 0.80651 to 0.82087, saving model to ./ak_resnet_model_24138194854.h5
54/54 [==============================] - 9s 161ms/step - loss: 0.3151 - precision: 0.9076 - val_loss: 0.3887 - val_precision: 0.8209
Epoch 4/50
54/54 [==============================] - ETA: 0s - loss: 0.2806 - precision: 0.9218
Epoch 4: val_precision improved from 0.82087 to 0.85000, saving model to ./ak_resnet_model_24138194854.

In [7]:
def dl_predict_gs(image_path, model, region_threshold=2, original_size=384, grid_size=2, save_points=False):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    seeds = lmb.find_seed_points(image)
    masks = [lmb.flood_fill(image, seed, lower=50, upper=50) for seed in seeds]
    composites = [lmb.overlay_mask_and_original(image, mask) for mask in masks]
    crops = [lmb.crop_100x100_from_point(composite, seed, adaptive_pad_color=False) for composite, seed in zip(composites, seeds)]

    positive_points = []
    negative_points = []
    cropidx = 0
    for crop in crops:
        crop = cv2.cvtColor(crop, cv2.COLOR_GRAY2RGB)
        # normalize between 0 and 1
        crop = crop / 255.
        crop = np.expand_dims(crop, axis=0)
        prediction = model.predict(crop, verbose=0)
        if prediction > 0.5:
            positive_points.append(seeds[cropidx])
        else:
            negative_points.append(seeds[cropidx])
        cropidx += 1

    if len(positive_points) >= region_threshold:
        region_results = gs.get_num_of_regions(original_size, grid_size, positive_points)
        if len(region_results) - region_results.count(0) >= region_threshold:
            if save_points:
                return "positive", positive_points, negative_points
            return "positive"
        else:
            if save_points:
                return "negative", positive_points, negative_points
            return "negative"
    else:
        if save_points:
            return "negative", positive_points, negative_points
        return "negative"

def save_img_with_mark(fp_path, original_path, positive_points, negative_points):
    # redo classification and save marked image
    image = cv2.imread(original_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    for positive_point in positive_points:
        image = cv2.circle(image, (positive_point[0], positive_point[1]), 7, (86, 247, 37), 1)
    for negative_point in negative_points:
        image = cv2.circle(image, (negative_point[0], negative_point[1]), 7, (218, 44, 245), 1)
    cv2.imwrite(fp_path + original_path.split('/')[-1], image)



In [9]:
# Patient-level Prediction
POSITIVE_THRESH = 0.3
best_model = load_model(best_model_file)
val_patients = set([path.split('/')[-1].split('_')[0] for path in val_paths])
patient_info = pws.get_patient_info(positive_directory, negative_directory)
total = len(val_paths)
FPDIR = './false_positives/'
FNDIR = './false_negatives/'
if not os.path.exists(FPDIR):
    os.makedirs(FPDIR)
else:
    shutil.rmtree(FPDIR)
    os.makedirs(FPDIR)
if not os.path.exists(FNDIR):
    os.makedirs(FNDIR)
else:
    shutil.rmtree(FNDIR)
    os.makedirs(FNDIR)

tp = 0
tn = 0
fp = 0
fn = 0
itp = 0
itn = 0
ifp = 0
ifn = 0
total_predictions = 0
for patient_id in val_patients:
    label = patient_info[patient_id][1]
    patient_image_paths = [path for path in val_paths if path.split('/')[-1].split('_')[0] == patient_id]
    patient_image_wise_results = []
    pathidx = 0

    for patient_image_path in patient_image_paths:
        print(f"Processing patient {patient_id} image {pathidx} of {len(patient_image_paths)}, Total {total_predictions}/{total}", end="\r")

        result, pos_points, neg_points = dl_predict_gs(patient_image_path, best_model, region_threshold=3, original_size=384, grid_size=2, save_points=True)

        if result == "positive" and label == "positive":
            itp += 1
        elif result == "negative" and label == "negative":
            itn += 1
        elif result == "positive" and label == "negative":
            save_img_with_mark(FPDIR, patient_image_path, pos_points, neg_points)
            ifp += 1
        elif result == "negative" and label == "positive":
            save_img_with_mark(FNDIR, patient_image_path, pos_points, neg_points)
            ifn += 1

        patient_image_wise_results.append(result)
        pathidx += 1
        total_predictions += 1

    if patient_image_wise_results.count("positive") >= len(patient_image_wise_results) * POSITIVE_THRESH:
        prediction_pw = "positive"
    else:
        prediction_pw = "negative"
    print("Patient ID:", patient_id, "Prediction:", prediction_pw, "Actual:", label)

    if prediction_pw == "positive" and label == "positive":
        tp += 1
        print(f"True Positive, {patient_image_wise_results.count('positive')} positive out of {len(patient_image_wise_results)}")
    elif prediction_pw == "negative" and label == "negative":
        tn += 1
        print(f"True Negative, {patient_image_wise_results.count('positive')} positive out of {len(patient_image_wise_results)}")
    elif prediction_pw == "positive" and label == "negative":
        fp += 1
        print(f"False Positive, {patient_image_wise_results.count('positive')} positive out of {len(patient_image_wise_results)}")
    elif prediction_pw == "negative" and label == "positive":
        fn += 1
        print(f"False Negative, {patient_image_wise_results.count('positive')} positive out of {len(patient_image_wise_results)}")


Patient ID: 104 Prediction: negative Actual: positive
False Negative, 0 positive out of 6
Patient ID: 30 Prediction: negative Actual: negative5
True Negative, 13 positive out of 106
Patient ID: 23 Prediction: negative Actual: negative
True Negative, 1 positive out of 75
Patient ID: 42 Prediction: positive Actual: positive
True Positive, 48 positive out of 98
Patient ID: 22 Prediction: negative Actual: negative
True Negative, 0 positive out of 1
Patient ID: 11 Prediction: negative Actual: negative5
True Negative, 0 positive out of 138
Patient ID: 70 Prediction: negative Actual: positive
False Negative, 4 positive out of 46
Patient ID: 50 Prediction: negative Actual: negative
True Negative, 0 positive out of 7
Patient ID: 67 Prediction: positive Actual: positive
True Positive, 12 positive out of 25
Patient ID: 71 Prediction: negative Actual: positive
False Negative, 0 positive out of 23
Patient ID: 107 Prediction: negative Actual: positive5
False Negative, 17 positive out of 126
Patient 

In [10]:
print("TP:", tp, "TN:", tn, "FP:", fp, "FN:", fn)
accuracy = (tp + tn) / (tp + tn + fp + fn)
print("Accuracy:", accuracy)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * precision * recall / (precision + recall)

print("Image-Wise TP:", itp, "TN:", itn, "FP:", ifp, "FN:", ifn)
iaccuracy = (itp + itn) / (itp + itn + ifp + ifn)
print("Accuracy:", iaccuracy)
iprecision = itp / (itp + ifp)
irecall = itp / (itp + ifn)
if1 = 2 * iprecision * irecall / (iprecision + irecall)
print(f"|{iaccuracy:.3f}|{if1:.3f}|{iprecision:.3f}|{irecall:.3f}|{itp}|{itn}|{ifp}|{ifn}|{accuracy:.3f}|{precision:.3f}|{recall:.3f}|{tp}|{tn}|{fp}|{fn}||")

TP: 2 TN: 6 FP: 0 FN: 4
Accuracy: 0.6666666666666666
Image-Wise TP: 81 TN: 327 FP: 14 FN: 243
Accuracy: 0.6135338345864662
|0.614|0.387|0.853|0.250|81|327|14|243|0.667|1.000|0.333|2|6|0|4||
